**Install Package**

In [242]:
!pip install textblob
!pip install sastrawi
!pip install emoji
!pip install PySastrawi

**Import Library**

In [243]:
import os
import pandas as pd
import re
import string
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import datetime
from datetime import timedelta
import numpy as np
import emoji
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

**Melakukan ekstraksi Data, Time, Contact dan Message**

In [244]:
# Extract date time
def date_time(s):
    pattern='^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
    result=re.match(pattern, s)
    if result:
        return True
    return False 

# Extract contacts
def find_contact(s):
    s=s.split(":")
    if len(s)>1:
        return True
    else:
        return False
    
# Extract Message
def getMassage(line):
    splitline=line.split(' - ')
    datetime= splitline[0];
    date, time= datetime.split(', ')
    message=" ".join(splitline[1:])
    
    if find_contact(message):
        splitmessage=message.split(": ")
        author=splitmessage[0]
        message=splitmessage[1]
    else:
        author=None
    return date, time, author, message

**Import Data text**

In [245]:
data=[]
conversation='Data5.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    messageBuffer=[]
    date, time, author= None, None, None
    while True:
        line=fp.readline()
        if not line:
            break
        line=line.strip()
        if date_time(line):
            if len(messageBuffer) >0:
                data.append([date, time, author, ''.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message=getMassage(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)

**Buat kolom baru dengan nama Date, Time, Author, Message**

In [246]:
dfwa = pd.DataFrame(data, columns=['Date', 'Time', 'Author', 'Message'])
dfwa

,Date,Time,Author,Message
0,8/29/18,11:02 AM,None,"Zuhelmi created group ""ASESOR BAN S/M PROV. RIAU"""
1,8/29/18,11:02 AM,None,You were added
2,12/18/20,2:49 PM,+62 812-7556-7884,https://www.idntimes.com/life/inspiration/days...
3,12/19/20,6:12 AM,Hidir Asesor,*Pribumi jualan di emperan Alfamart.*Suatu sor...
4,12/19/20,6:42 AM,+62 813-6552-5099,Sangar bermamfaat
...,...,...,...,...
1195,3/20/22,11:51 AM,Hidir Asesor,Numpang iklan .punya anak
1196,3/20/22,11:51 AM,Hidir Asesor,<Media omitted>
1197,3/21/22,6:43 AM,+62 812-7556-7884,https://www.cnbcindonesia.com/news/20220319074...
1198,3/21/22,6:43 AM,+62 812-7556-7884,<Media omitted>


**Preprocessing**

In [247]:
# Menghapus baris yang berisi Media omitted 

dfwa = dfwa[dfwa['Message'] != '<Media omitted>']
dfwa = dfwa.dropna()
dfwa

,Date,Time,Author,Message
2,12/18/20,2:49 PM,+62 812-7556-7884,https://www.idntimes.com/life/inspiration/days...
3,12/19/20,6:12 AM,Hidir Asesor,*Pribumi jualan di emperan Alfamart.*Suatu sor...
4,12/19/20,6:42 AM,+62 813-6552-5099,Sangar bermamfaat
5,12/19/20,6:43 AM,+62 813-7168-0541,Menjadi tersentuh membacanya
6,12/19/20,8:56 AM,Defrizon Siak,"Jika merasa sangat bermanfaat dan tersentuh, m..."
...,...,...,...,...
1180,3/11/22,9:21 AM,+62 852-7899-6666,Mentri perpanjangan tangan presiden....Supaya ...
1181,3/11/22,9:24 AM,Hidir Asesor,Hahaha... kita kesusahan..mrk yg jual ke LN
1188,3/17/22,2:50 PM,+62 812-7556-7884,https://m.facebook.com/story.php?story_fbid=34...
1195,3/20/22,11:51 AM,Hidir Asesor,Numpang iklan .punya anak


In [248]:
# Casefolding pada kolom Message

def casefolding(Message):
  Message = Message.lower()
  Message = Message.strip(" ")
  Message = re.sub(r'[/|@|?|$|.|!_:")(-+,]', '', Message)
  Message = re.sub(r'http\S+', '', Message)
  Message = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", Message).split())
  return Message
dfwa['Message'] = dfwa['Message'].apply(casefolding)
dfwa.head(100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Date,Time,Author,Message
2,12/18/20,2:49 PM,+62 812-7556-7884,
3,12/19/20,6:12 AM,Hidir Asesor,pribumi jualan di emperan alfamartsuatu sore k...
4,12/19/20,6:42 AM,+62 813-6552-5099,sangar bermamfaat
5,12/19/20,6:43 AM,+62 813-7168-0541,menjadi tersentuh membacanya
6,12/19/20,8:56 AM,Defrizon Siak,jika merasa sangat bermanfaat dan tersentuh ma...
...,...,...,...,...
137,1/12/21,11:17 AM,+62 813-6576-0480,this message was deleted
138,1/12/21,11:19 AM,+62 813-6576-0480,innalilahiwainalilahirojiunsemoga husnul khotimah
139,1/12/21,11:28 AM,Susandri,innalillahi wainnailaihi rojiunturut berduka c...
141,1/12/21,11:44 AM,+62 813-6527-6661,semoga alm diampunkan dosanyadilapangkan kubur...


In [249]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [250]:
# Import Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [251]:
from nltk import word_tokenize

In [252]:
stop_factory = StopWordRemoverFactory().get_stop_words()
more_stopwords = [
                  'yg', 'utk', 'cuman', 'deh', 'Btw', 'tapi', 'gua', 'gue', 'lo', 'lu', 'blm',
                  'kalo', 'trs', 'trus', 'jd', 'nih', 'ntar', 'nya', 'lg', 'gk', 'dpt', 'rp',
                  'dr', 'kpn', 'kok', 'kyk', 'dong', 'yah', 'ooh', 'ya', 'ga', 'km', 'eh', 'ga',
                  'sih', 'eh', 'bang', 'br', 'rp', 'jt', 'kan', 'gpp', 'sm', 'usah', 'mas', 'tk'
                  'sob', 'jg', 'haha', 'sy', 'tdk', 'wkwkwk', 'lbh', 'org', 'gtu', 'jgn', 'krn', 'udah'
]
data = stop_factory + more_stopwords 

dictionary = ArrayDictionary(data)
str = StopWordRemover(dictionary)

print(data)

['a', 'ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'arti', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'b', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagainamakah', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasannya', 'bahwasanya', 'baik', 'baiklah', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumla

In [253]:
dfwa = dfwa[dfwa['Message']!='']
dfwa.head()

,Date,Time,Author,Message
3,12/19/20,6:12 AM,Hidir Asesor,pribumi jualan di emperan alfamartsuatu sore k...
4,12/19/20,6:42 AM,+62 813-6552-5099,sangar bermamfaat
5,12/19/20,6:43 AM,+62 813-7168-0541,menjadi tersentuh membacanya
6,12/19/20,8:56 AM,Defrizon Siak,jika merasa sangat bermanfaat dan tersentuh ma...
7,12/19/20,8:58 AM,+62 812-7676-935,ya pak selagi ada di warung tetangga bagitu ni...


In [254]:
def remove_punct(Message):
  Message = " ".join([char for char in Message if char not in string.punctuation])
  return Message
dfwa['Message'] = dfwa['Message'].apply(lambda x: remove_punct(x))  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [255]:
dfwa.head()

,Date,Time,Author,Message
3,12/19/20,6:12 AM,Hidir Asesor,p r i b u m i j u a l a n d i e m p e r ...
4,12/19/20,6:42 AM,+62 813-6552-5099,s a n g a r b e r m a m f a a t
5,12/19/20,6:43 AM,+62 813-7168-0541,m e n j a d i t e r s e n t u h m e m b a ...
6,12/19/20,8:56 AM,Defrizon Siak,j i k a m e r a s a s a n g a t b e r m ...
7,12/19/20,8:58 AM,+62 812-7676-935,y a p a k s e l a g i a d a d i w a ...


In [256]:
# Reset Index
dfwa = dfwa.reset_index(drop=True)
dfwa.head()

,Date,Time,Author,Message
0,12/19/20,6:12 AM,Hidir Asesor,p r i b u m i j u a l a n d i e m p e r ...
1,12/19/20,6:42 AM,+62 813-6552-5099,s a n g a r b e r m a m f a a t
2,12/19/20,6:43 AM,+62 813-7168-0541,m e n j a d i t e r s e n t u h m e m b a ...
3,12/19/20,8:56 AM,Defrizon Siak,j i k a m e r a s a s a n g a t b e r m ...
4,12/19/20,8:58 AM,+62 812-7676-935,y a p a k s e l a g i a d a d i w a ...


In [257]:
#Save file bersih
dfwa.to_csv('DataBersih1.csv', encoding='utf-8', index=False)

In [258]:
dfwa.drop(dfwa.columns[[0,1,2]], axis = 1, inplace = True)
dfwa.head()

,Message
0,p r i b u m i j u a l a n d i e m p e r ...
1,s a n g a r b e r m a m f a a t
2,m e n j a d i t e r s e n t u h m e m b a ...
3,j i k a m e r a s a s a n g a t b e r m ...
4,y a p a k s e l a g i a d a d i w a ...


In [259]:
#Save file bersih
dfwa.to_csv('DataBersih2.csv', encoding='utf-8', index=False)

In [260]:
!pip install googletrans==3.1.0a0

In [261]:
import pandas as pd
import googletrans
from googletrans import Translator
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [262]:
dfwa = pd.read_csv('DataBersih2.csv')
dfwa.head(10)

,Message
0,p r i b u m i j u a l a n d i e m p e r ...
1,s a n g a r b e r m a m f a a t
2,m e n j a d i t e r s e n t u h m e m b a ...
3,j i k a m e r a s a s a n g a t b e r m ...
4,y a p a k s e l a g i a d a d i w a ...
5,a l j a z e e r a c h i n a b o r o n g ...
6,k a r e n a s i n o v a c u d a h a b i ...
7,k h a w a t i r n y a k i t a d i b o d ...
8,m o u s u d a h d i t a n d a t a n g a n ...
9,c h i n a s e n d i r i b l m m e m a k ...


**Menerjemahkan kolom Message dari Bahasa Inggris ke Indonesia**

In [263]:
translator = Translator()
translations={}
for column in dfwa.columns:
  unique_elements = dfwa[column].unique()
  for element in unique_elements:
    translations[element] = translator.translate(element).text
translations    

{'1   m u h a r a m   t e t a p   t g l   1 0   a g u s t u s   t d k   b e r u b a h   y g   b e r u b a h   h a n y a   h a r i   l i b u r n y a   s a j a   p a k': '1   m u h a r a m   t e t a p   t g l   1 0   a g u s t u s   t d k   b e r u b a h   y g   b e r u b a h   h a n y a   h a r i   l i b u r n y a   s a j a   p a k',
 '1   p e k a l o n g a n 3   p u r w o k e r t o 4   b a n y u w a n g i 5   p u r w o d a d i 7   t r e n g g a l e k 8   s u r a b a y a 9   p o n t i a n a k 1 1   m e r a u k e 1 2   b u k i t t i n g g i 1 4   m a g e l a n g 1 5   p u r w a k a r t a 1 6   j o g y a k a r t a 1 7   p a n g k a l a r a y a 2 2   d e n o a s a r 2 3   s u r a k a r t a 2 6   j a y a p u r a 3 0   b a n d a r   l a m p u n g 3 3   m a k a s a r 3 4   j a k a r t a 3 6   m a d i u n 3 7   b a n j a r m a s i n 3 8   b a n d a   a c e h 3 9   m o j o k e r t o 4 1   b a l i k p a p a n 4 2   a t a m b u a 4 3   s e m a r a n g 4 5   k a r a n g a n y a r 4 7   b o n d o w

In [264]:
translator = Translator()

dfwa['Message_English'] = dfwa['Message'].apply(lambda x: translator.translate(x, dest='en').text)

In [265]:
dfwa

,Message,Message_English
0,p r i b u m i j u a l a n d i e m p e r ...,p r i b u m i j u a l a n d i e m p e r ...
1,s a n g a r b e r m a m f a a t,s a n g a r b e r m a m f a a t
2,m e n j a d i t e r s e n t u h m e m b a ...,m e n j a d i t e r s e n t u h m e m b a ...
3,j i k a m e r a s a s a n g a t b e r m ...,j i k a m e r a s a s a n g a t b e r m ...
4,y a p a k s e l a g i a d a d i w a ...,y a p a k s e l a g i a d a d i w a ...
...,...,...
569,n u m p a n g i k l a n,n u m p a n g i k l a n
570,m a n t a b b b b b,m a n t a b b b b b
571,m e n t r i p e r p a n j a n g a n t a n ...,m e n t r i p e r p a n j a n g a n t a n ...
572,h a h a h a k i t a k e s u s a h a n m r ...,h a h a h a k i t a k e s u s a h a n m r ...


In [266]:
#Save file bersih
dfwa.to_csv('translate_DataBersih.csv', encoding='utf-8', index=False)

In [267]:
dfwa = pd.read_csv('translate_DataBersih.csv')
dfwa.head(10)

,Message,Message_English
0,p r i b u m i j u a l a n d i e m p e r ...,p r i b u m i j u a l a n d i e m p e r ...
1,s a n g a r b e r m a m f a a t,s a n g a r b e r m a m f a a t
2,m e n j a d i t e r s e n t u h m e m b a ...,m e n j a d i t e r s e n t u h m e m b a ...
3,j i k a m e r a s a s a n g a t b e r m ...,j i k a m e r a s a s a n g a t b e r m ...
4,y a p a k s e l a g i a d a d i w a ...,y a p a k s e l a g i a d a d i w a ...
5,a l j a z e e r a c h i n a b o r o n g ...,a l j a z e e r a c h i n a b o r o n g ...
6,k a r e n a s i n o v a c u d a h a b i ...,k a r e n a s i n o v a c u d a h a b i ...
7,k h a w a t i r n y a k i t a d i b o d ...,k h a w a t i r n y a k i t a d i b o d ...
8,m o u s u d a h d i t a n d a t a n g a n ...,m o u s u d a h d i t a n d a t a n g a n ...
9,c h i n a s e n d i r i b l m m e m a k ...,c h i n a s e n d i r i b l m m e m a k ...


**Melakukan proses Sentiment Analysis untuk memberikan nilai Positif, Neutral dan Negatif pada kolom Message**

**Lexicon Based**

In [268]:
!pip install VaderSentiment

In [269]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [270]:
scores = [analyser.polarity_scores(x) for x in dfwa['Message']]
print(scores)
dfwa['Compound_Score'] = [x['compound'] for x in scores]

[{'neg': 0.0, 'neu': 0.89, 'pos': 0.11, 'compound': 0.9999}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 0.903, 'pos': 0.097, 'compound': 0.9776}, {'neg': 0.0, 'neu': 0.938, 'pos': 0.062, 'compound': 0.4588}, {'neg': 0.0, 'neu': 0.924, 'pos': 0.076, 'compound': 0.9517}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 0.872, 'pos': 0.128, 'compound': 0.9818}, {'neg': 0.0, 'neu': 0.914, 'pos': 0.086, 'compound': 0.4588}, {'neg': 0.0, 'neu': 0.943, 'pos': 0.057, 'compound': 0.7184}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 0.857, 'pos': 0.143, 'compound': 0.7184}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 0.844, 'pos': 0.156, 'compound': 0.9001}, {'neg': 0.0, 'neu': 0.857, 'pos': 0.143, 'compound': 0.4588}, {'neg': 0.0, 'neu': 0.955, 'pos': 0.045, 'compound'

In [271]:
dfwa.head(100)

,Message,Message_English,Compound_Score
0,p r i b u m i j u a l a n d i e m p e r ...,p r i b u m i j u a l a n d i e m p e r ...,0.9999
1,s a n g a r b e r m a m f a a t,s a n g a r b e r m a m f a a t,0.0000
2,m e n j a d i t e r s e n t u h m e m b a ...,m e n j a d i t e r s e n t u h m e m b a ...,0.0000
3,j i k a m e r a s a s a n g a t b e r m ...,j i k a m e r a s a s a n g a t b e r m ...,0.9776
4,y a p a k s e l a g i a d a d i w a ...,y a p a k s e l a g i a d a d i w a ...,0.4588
...,...,...,...
95,a m i n,a m i n,0.0000
96,t h i s m e s s a g e w a s d e l e t e d,t h i s m e s s a g e w a s d e l e t e d,0.4588
97,y g l a g i p a n a s,y g l a g i p a n a s,0.4588
98,b e r a n i n d a k r e z i m,b e r a n i n d a k r e z i m,0.0000


In [272]:
# Compound Score lexicon based dengan Positif, Netral dan Negatif
dfwa.loc[dfwa['Compound_Score'] < 0, 'Sentiments'] = 'Negatif'
dfwa.loc[dfwa['Compound_Score'] == 0, 'Sentiments'] = 'Netral'
dfwa.loc[dfwa['Compound_Score'] > 0, 'Sentiments'] = 'Positif'

dfwa.head(100)

,Message,Message_English,Compound_Score,Sentiments
0,p r i b u m i j u a l a n d i e m p e r ...,p r i b u m i j u a l a n d i e m p e r ...,0.9999,Positif
1,s a n g a r b e r m a m f a a t,s a n g a r b e r m a m f a a t,0.0000,Netral
2,m e n j a d i t e r s e n t u h m e m b a ...,m e n j a d i t e r s e n t u h m e m b a ...,0.0000,Netral
3,j i k a m e r a s a s a n g a t b e r m ...,j i k a m e r a s a s a n g a t b e r m ...,0.9776,Positif
4,y a p a k s e l a g i a d a d i w a ...,y a p a k s e l a g i a d a d i w a ...,0.4588,Positif
...,...,...,...,...
95,a m i n,a m i n,0.0000,Netral
96,t h i s m e s s a g e w a s d e l e t e d,t h i s m e s s a g e w a s d e l e t e d,0.4588,Positif
97,y g l a g i p a n a s,y g l a g i p a n a s,0.4588,Positif
98,b e r a n i n d a k r e z i m,b e r a n i n d a k r e z i m,0.0000,Netral


Memberikan value pada kolom Sentiments

In [273]:
# Compound Score lexicon based dengan Positif, Netral dan Negatif
dfwa.loc[dfwa['Sentiments'] == 'Positif', 'Skor'] = '1'
dfwa.loc[dfwa['Sentiments'] == 'Netral', 'Skor'] = '0'
dfwa.loc[dfwa['Sentiments'] == 'Negatif', 'Skor'] = '-1'
dfwa.head()

,Message,Message_English,Compound_Score,Sentiments,Skor
0,p r i b u m i j u a l a n d i e m p e r ...,p r i b u m i j u a l a n d i e m p e r ...,0.9999,Positif,1
1,s a n g a r b e r m a m f a a t,s a n g a r b e r m a m f a a t,0.0000,Netral,0
2,m e n j a d i t e r s e n t u h m e m b a ...,m e n j a d i t e r s e n t u h m e m b a ...,0.0000,Netral,0
3,j i k a m e r a s a s a n g a t b e r m ...,j i k a m e r a s a s a n g a t b e r m ...,0.9776,Positif,1
4,y a p a k s e l a g i a d a d i w a ...,y a p a k s e l a g i a d a d i w a ...,0.4588,Positif,1


In [274]:
#Save file bersih
dfwa.to_csv('translate_DataBersih2.csv', encoding='utf-8', index=False)

In [275]:
import pandas as pd
def load_data():
  dfw = pd.read_csv('translate_DataBersih2.csv')
  return dfwa

In [276]:
dfwa = load_data()
dfwa.head()

,Message,Message_English,Compound_Score,Sentiments,Skor
0,p r i b u m i j u a l a n d i e m p e r ...,p r i b u m i j u a l a n d i e m p e r ...,0.9999,Positif,1
1,s a n g a r b e r m a m f a a t,s a n g a r b e r m a m f a a t,0.0000,Netral,0
2,m e n j a d i t e r s e n t u h m e m b a ...,m e n j a d i t e r s e n t u h m e m b a ...,0.0000,Netral,0
3,j i k a m e r a s a s a n g a t b e r m ...,j i k a m e r a s a s a n g a t b e r m ...,0.9776,Positif,1
4,y a p a k s e l a g i a d a d i w a ...,y a p a k s e l a g i a d a d i w a ...,0.4588,Positif,1


In [277]:
dfwa.shape

(574, 5)

In [278]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import pandas as pd

factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

dfwa = pd.read_csv("translate_DataBersih2.csv")
for i, Sentiments in enumerate (dfwa['Sentiments']):
    stop = stopword.remove(Sentiments)
    print(i,stop + '\n')

0 Positif

1 Netral

2 Netral

3 Positif

4 Positif

5 Positif

6 Netral

7 Positif

8 Positif

9 Positif

10 Netral

11 Netral

12 Positif

13 Netral

14 Positif

15 Positif

16 Positif

17 Netral

18 Netral

19 Positif

20 Positif

21 Positif

22 Positif

23 Positif

24 Positif

25 Positif

26 Positif

27 Positif

28 Positif

29 Positif

30 Positif

31 Positif

32 Positif

33 Netral

34 Positif

35 Positif

36 Positif

37 Positif

38 Positif

39 Positif

40 Positif

41 Positif

42 Netral

43 Positif

44 Positif

45 Netral

46 Positif

47 Netral

48 Positif

49 Netral

50 Positif

51 Positif

52 Positif

53 Netral

54 Positif

55 Netral

56 Positif

57 Netral

58 Positif

59 Netral

60 Positif

61 Netral

62 Positif

63 Positif

64 Positif

65 Positif

66 Positif

67 Netral

68 Netral

69 Positif

70 Positif

71 Positif

72 Positif

73 Positif

74 Positif

75 Positif

76 Positif

77 Positif

78 Positif

79 Positif

80 Positif

81 Positif

82 Positif

83 Positif

84 Positif

85 Positif

In [279]:
"""s = pd.value_counts(dfwa['Sentiments'])
ax = s.plot.bar()
n = len(dfwa.index)
print(n)
for p in ax.patches:
  ax.annotate(str(round(p.get_height() / n * 100, 2)) + '%', (p.get_x() * 1.005, p.get_height() * 1.005))"""

"s = pd.value_counts(dfwa['Sentiments'])\nax = s.plot.bar()\nn = len(dfwa.index)\nprint(n)\nfor p in ax.patches:\n  ax.annotate(str(round(p.get_height() / n * 100, 2)) + '%', (p.get_x() * 1.005, p.get_height() * 1.005))"

In [280]:
"""wordcloud = WordCloud(width = 800, height = 800, background_color = 'black', max_words = 1000,
                      min_font_size= 20).generate(str(dfwa))
# Plot the word cloud
fig = plt.figure(figsize = (8,8), facecolor = None)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()"""

"wordcloud = WordCloud(width = 800, height = 800, background_color = 'black', max_words = 1000,\n                      min_font_size= 20).generate(str(dfwa))\n# Plot the word cloud\nfig = plt.figure(figsize = (8,8), facecolor = None)\nplt.imshow(wordcloud)\nplt.axis('off')\nplt.show()"

**Menggunakan algoritma LSTM untuk mendapatkan nilai akurasi**

**LSTM**

In [281]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

In [282]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix

train_df, test_df = train_test_split(dfwa, test_size = 0.2, random_state = 42)
print("Training data size : ", train_df.shape)
print("Test data size : ", test_df.shape)

Training data size :  (459, 5)
Test data size :  (115, 5)


In [283]:
top_words = 1000
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(train_df['Message'])
list_tokenized_train = tokenizer.texts_to_sequences(train_df['Message'])

max_review_length = 200
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
y_train = train_df['Skor']

In [284]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # optimizer = adam
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 200, 32)           32000     
                                                                 
 lstm_6 (LSTM)               (None, 100)               53200     
                                                                 
 dense_11 (Dense)            (None, 1)                 101       
                                                                 
Total params: 85,301
Trainable params: 85,301
Non-trainable params: 0
_________________________________________________________________


In [285]:
model.fit(X_train,y_train, epochs=20, batch_size=64, validation_split=0.2)

Epoch 1/20
6/6 [==============================] - 5s 476ms/step - loss: 0.6779 - accuracy: 0.7193 - val_loss: 0.6525 - val_accuracy: 0.7283
Epoch 2/20
6/6 [==============================] - 2s 399ms/step - loss: 0.5814 - accuracy: 0.7548 - val_loss: 0.5149 - val_accuracy: 0.7283
Epoch 3/20
6/6 [==============================] - 3s 466ms/step - loss: 0.4123 - accuracy: 0.7602 - val_loss: 0.4122 - val_accuracy: 0.8043
Epoch 4/20
6/6 [==============================] - 2s 418ms/step - loss: 0.3776 - accuracy: 0.8229 - val_loss: 0.3969 - val_accuracy: 0.8261
Epoch 5/20
6/6 [==============================] - 2s 396ms/step - loss: 0.3629 - accuracy: 0.8474 - val_loss: 0.4041 - val_accuracy: 0.8261
Epoch 6/20
6/6 [==============================] - 2s 390ms/step - loss: 0.3473 - accuracy: 0.8529 - val_loss: 0.3849 - val_accuracy: 0.8370
Epoch 7/20
6/6 [==============================] - 3s 437ms/step - loss: 0.3267 - accuracy: 0.8501 - val_loss: 0.3656 - val_accuracy: 0.8261
Epoch 8/20
6/6 [====

In [286]:
list_tokenized_test = tokenizer.texts_to_sequences(test_df['Message'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = test_df['Skor']
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)

Accuracy of the model :  0.9304347826086956
F1-score:  0.9545454545454545
Confusion matrix:


array([[23,  2],
       [ 6, 84]])

**Menggunakan algoritma RNN untuk mendapatkan nilai akurasi, lebih kurang menyempurnakan nilai akurasi pada algoritma LSTM yang sebelumnya**

**RNN**

In [287]:
model0 = Sequential()
model0.add(layers.Embedding(top_words, 32, input_length=max_review_length))
model0.add(layers.SimpleRNN(100))
model0.add(layers.Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # optimizer = adam
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 200, 32)           32000     
                                                                 
 lstm_6 (LSTM)               (None, 100)               53200     
                                                                 
 dense_11 (Dense)            (None, 1)                 101       
                                                                 
Total params: 85,301
Trainable params: 85,301
Non-trainable params: 0
_________________________________________________________________


In [288]:
model.fit(X_train,y_train, epochs=20, batch_size=64, validation_split=0.2)

Epoch 1/20
6/6 [==============================] - 4s 356ms/step - loss: 0.1078 - accuracy: 0.9619 - val_loss: 0.1386 - val_accuracy: 0.9457
Epoch 2/20
6/6 [==============================] - 1s 249ms/step - loss: 0.0909 - accuracy: 0.9700 - val_loss: 0.1144 - val_accuracy: 0.9457
Epoch 3/20
6/6 [==============================] - 1s 244ms/step - loss: 0.0730 - accuracy: 0.9864 - val_loss: 0.1072 - val_accuracy: 0.9457
Epoch 4/20
6/6 [==============================] - 1s 245ms/step - loss: 0.0601 - accuracy: 0.9809 - val_loss: 0.0881 - val_accuracy: 0.9565
Epoch 5/20
6/6 [==============================] - 2s 408ms/step - loss: 0.0440 - accuracy: 0.9891 - val_loss: 0.0769 - val_accuracy: 0.9674
Epoch 6/20
6/6 [==============================] - 2s 393ms/step - loss: 0.0418 - accuracy: 0.9809 - val_loss: 0.0717 - val_accuracy: 0.9674
Epoch 7/20
6/6 [==============================] - 3s 451ms/step - loss: 0.0342 - accuracy: 0.9918 - val_loss: 0.0876 - val_accuracy: 0.9565
Epoch 8/20
6/6 [====

In [289]:
list_tokenized_test = tokenizer.texts_to_sequences(test_df['Message'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = test_df['Skor']
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)

Accuracy of the model :  0.9826086956521739
F1-score:  0.9888888888888889
Confusion matrix:


array([[24,  1],
       [ 1, 89]])

**Ada peningkatan nilai akurasi yang sebelumnya  0.9304347826086956 menjadi 0.9826086956521739 setelah menggunakan metode hybrid dengan RNN**